<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 4.2: FIRRTL AST Traversal

**Prev: [Introduction to FIRRTL](4.1_firrtl_ast.ipynb)**<br>
**Next: [Common Pass Idioms](4.3_firrtl_common_idioms.ipynb)**

### Understanding IR node children

Firrtl 패스를 작성하려면 일반적으로 정보를 수집하거나 IR 노드를 새 IR 노드로 교체하기 위해 Firrtl 데이터 구조를 안내하는 작성 기능이 필요합니다.

IR 데이터 구조는 트리이며, 각 IR 노드는 몇 개의 자식 노드를 가질 수 있습니다(이는 차례로 더 많은 자식 노드 등을 가질 수 있음). 자식이 없는 IR 노드를 잎이라고 합니다.

다른 IR 노드는 다른 자식 유형을 가질 수 있습니다. 다음 표는 각 IR 노드 유형에 대해 가능한 자식 유형을 보여줍니다.

```
+------------+-----------------------------+
|    Node    |          Children           |
+------------+-----------------------------+
| Circuit    | DefModule                   |
| DefModule  | Port, Statement             |
| Port       | Type, Direction             |
| Statement  | Statement, Expression, Type |
| Expression | Expression, Type            |
| Type       | Type, Width                 |
| Width      |                             |
| Direction  |                             |
+------------+-----------------------------+
```

### The map function

'회로'를 가로지르는 함수를 작성하려면 먼저 함수형 프로그래밍 개념인 '맵'을 이해해야 합니다.

#### Understanding Seq.map
스칼라 문자열 시퀀스는 루트 노드 `Seq`와 자식 노드 `"a"`, `"b"`, `"c"`가 있는 트리로 나타낼 수 있습니다.
```scala
val s = Seq("a", "b", "c")
```
```
    Seq
 /   |   \
"a" "b" "c"
```

String 인수 `x`가 주어지면 `x`를 자신과 연결하는 함수 `f`를 정의한다고 가정합니다.
```scala
def f(x: String): String = x + x
```

`s.map`을 호출하여 s의 모든 자식에 `f`를 적용한 결과인 새 `Seq[String]`을 반환할 수 있습니다.
```scala
val s = Seq("a", "b", "c")
def f(x: String): String = x + x  // repeated declaration for clarity
val t = s.map(f)
println(t) // Seq("aa", "bb", "cc")
```
```
     Seq
 /    |    \
"aa" "bb" "cc"
```

#### Understanding Firrtl's map

우리는 이 "매핑" 아이디어를 사용하여 IR 노드에서 고유한 사용자 정의 `map` 메서드를 만듭니다. 1 + 1을 나타내는 'DoPrim' 표현식이 있다고 가정합니다. 이것은 루트 노드 'DoPrim'이 있는 표현식 트리로 나타낼 수 있습니다.
```
        DoPrim
     /          \
UIntValue    UIntValue
```

`Expression` 인수를 취하고 새로운 `Expression`을 반환하는 함수 `f`가 있는 경우 `DoPrim`과 같이 주어진 IR 노드의 모든 하위 `Expression`에 이를 "매핑"할 수 있습니다. 그러면 자식이 `DoPrim`의 모든 `Expression` 자식에 `f`를 적용한 결과인 다음과 같은 새 `DoPrim`이 반환됩니다.
```
        DoPrim
     /          \
f(UIntValue)    f(UIntValue)
```

때때로 IR 노드에는 여러 유형의 자식이 있습니다. 예를 들어 'Conditionally'에는 'Expression'과 'Statement' 자식이 모두 있습니다. 이 경우 맵은 해당 유형이 함수의 인수 유형(및 반환 값 유형)과 일치하는 하위 항목에만 해당 함수를 적용합니다.
```scala
val c = Conditionally(info, e, s1, s2) // e: Expression, s1, s2: Statement, info: FileInfo
def fExp(e: Expression): Expression = ...
def fStmt(s: Statement): Statement = ...
c.map(fExp)  // Conditionally(fExp(e), s1, s2)
c.map(fStmt) // Conditionally(e, fStmt(s1), fStmt(s2))
```

스칼라는 하나의 인수를 가진 함수를 호출할 때 `.`와 괄호를 삭제할 수 있는 "중위 표기법"을 가지고 있습니다. 종종 우리는 이러한 맵 함수를 중위 표기법으로 작성합니다.
```scala
c map fExp  // equivalent to c.map(fExp)
c map fStmt // equivalent to c.map(fStmt)
```

### Pre-order traversal

Firrtl 트리를 탐색하기 위해 'map'을 사용하여 관심 있는 모든 노드의 모든 자식을 방문하는 재귀 함수를 작성합니다.

디자인에 선언된 모든 레지스터의 이름을 수집하고 싶다고 가정합니다. 우리는 이것이 모든 `문`을 방문해야 한다는 것을 알고 있습니다. 그러나 일부 `Statement` 노드에는 자식 `Statement`가 있을 수 있습니다. 따라서 입력 인수가 'DefRegister'인지 확인하고 그렇지 않은 경우 입력 인수의 모든 'Statement' 자식에 'f'를 재귀적으로 적용하는 함수를 작성해야 합니다.

다음 함수 'f'는 설명된 함수와 유사하지만 레지스터 이름의 가변 해시 집합과 '문'이라는 두 가지 인수를 취합니다. 함수 커링을 사용하면 첫 번째 인수만 전달하여 원하는 유형 서명(`Statement=>Statement`)이 있는 새 함수를 반환할 수 있습니다.

```scala
def f(regNames: mutable.HashSet[String]())(s: Statement): Statement = s match {
  // If register, add name to regNames
  case r: DefRegister =>
    regNames += r.name
    r // Return argument unchanged (ok because DefRegister has no Statement children)
  // If not, apply f(regNames) to all children Statement
  case _ => s map f(regNames) // Note that f(regNames) is of type Statement=>Statement
}
```

이 패턴은 Firrtl에서 매우 일반적이며 재귀 함수가 자식 노드에 재귀적으로 적용하기 전에 원래 IR 노드에서 일치하기 때문에 "선주문 순회"라고 합니다.

### Post-order traversal

다음과 같이 "후위 순회"에서 이전 예제를 작성할 수 있습니다.

```scala
def f(regNames: mutable.HashSet[String]())(s: Statement): Statement = 
  // Not we immediately recurse to the children nodes, then match
  s map f(regName) match {
    // If register, add name to regNames
    case r: DefRegister =>
      regNames += r.name
      r // Return argument unchanged (ok because DefRegister has no Statement children)
    // If not, return s
    case _ => s // Note that all Statement children of s have had f(regNames) already applied
  }
```

순회 순서는 이 두 예제 간에 다르지만 이 사용 사례(및 다른 많은 사례)에서는 차이가 없습니다. 그러나 순회 순서가 중요할 때를 대비하여 백 포켓에 보관하는 중요한 도구입니다.